In [1]:
import os
import numpy as np 
from PIL import Image

In [2]:
def preprocess_image(image, height: int, width: int, all_channels: bool = True):
    image_res = image.resize((width, height), Image.ANTIALIAS)  # Resizes Image to given width/height

    if not all_channels:  # For models who process all R'G'B' components of the input image data
        image_res = image_res.convert("YCbCr")
        y, cb_img, cr_img = image_res.split()
        y_arr = np.asarray(y).astype(np.float32)
        y_arr /= 255.0
        y_arr = np.expand_dims(np.expand_dims(y_arr, 0), 0)
        return y_arr, cb_img, cr_img
    else:  # For models who process all R'G'B' components of the input image data
        image_res = image_res.convert("RGB")
        image_d = np.asarray(image_res).astype(
            np.float32)  # Transforms image to a [H,W,C] array format.i.e. (128,128,3)
        image_data = image_d.transpose([2, 0, 1])  # transpose to [C,H,W] format
        image_dat = np.expand_dims(image_data / 255, axis=0)
        return image_dat

In [3]:
def postprocess_image(image_arr, cb=None, cr=None):
    image_arr = image_arr[0]
    image_arr *= 255.0
    image_arr = image_arr.clip(0, 255)

    if cb is not None and cr is not None:  # For models who process all R'G'B' components of the input image data
        y = Image.fromarray(np.uint8(image_arr)[0], mode="L")
        cb = cb.resize((y.size[0], y.size[1]), Image.ANTIALIAS)
        cr = cr.resize((y.size[0], y.size[1]), Image.ANTIALIAS)
        post_img = Image.merge("YCbCr",
                               [y, cb, cr]).convert("RGB")
        return post_img
    else:  # For models who process all R'G'B' components of the input image data
        red, green, blue = image_arr
        red = Image.fromarray(np.uint8(red), mode="L")
        green = Image.fromarray(np.uint8(green), mode="L")
        blue = Image.fromarray(np.uint8(blue), mode="L")
        post_img = Image.merge("RGB",
                               [red, green, blue])
        return post_img


In [4]:
"""
The 'preprocess_image' function takes in 4 arguments: 
    image(PIL.Image.Image)- The image data in picture format;
    height(Int)- The resized height dimension
    width(Int)- The resized width dimension
    all_channels(Bool)- Optional argument. Specifies if the model processes all the image data channels or just one channel;
    
    if 'all_channels' is true, returns a [1, 3, height, width] array; 
        else returns a union tuple containing a [1,1,height,width] array, blue-difference, and red-difference component of the image data.

The 'postprocess_image' function takes in 3 arguments:
    image_arr(numpy.ndarray)- The processed image data in array format;
    cb(PIL.Image.Image)- Optional argument for the blue-difference component of image data. Use if model works on only one component of the image data; 
    cr(PIL.Image.Image)- Optional argument for the red-difference component of image data. Use if model works on only one component of the image data;
    
    returns the processed image data in picture format

""" 



"\nThe 'preprocess_image' function takes in 4 arguments: \n    image(PIL.Image.Image)- The image data in picture format;\n    height(Int)- The resized height dimension\n    width(Int)- The resized width dimension\n    all_channels(Bool)- Optional argument. Specifies if the model processes all the image data channels or just one channel;\n    \n    if 'all_channels' is true, returns a [1, 3, height, width] array; \n        else returns a union tuple containing a [1,1,height,width] array, blue-difference, and red-difference component of the image data.\n\nThe 'postprocess_image' function takes in 3 arguments:\n    image_arr(numpy.ndarray)- The processed image data in array format;\n    cb(PIL.Image.Image)- Optional argument for the blue-difference component of image data. Use if model works on only one component of the image data; \n    cr(PIL.Image.Image)- Optional argument for the red-difference component of image data. Use if model works on only one component of the image data;\n    \